In [1]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

!ln -s '/content/gdrive/MyDrive/VRDL_HW2' /WORK

Mounted at /content/gdrive/


# GIT

In [ ]:
%cd '/content/gdrive/My Drive/VRDL_HW2'
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd /content/gdrive/My Drive/VRDL_HW2/darknet

In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

# Create Data

### training data

In [ ]:
%cd /content/gdrive/MyDrive/VRDL_HW2/darknet

/content/gdrive/MyDrive/VRDL_HW2/darknet


In [ ]:
import glob
import pandas as pd
import numpy as np
json_file_pathlist = glob.glob("data/train/*.png")
print(json_file_pathlist)

In [ ]:
len(json_file_pathlist)

In [ ]:
df = pd.DataFrame({"image_path": json_file_pathlist})
df.head()

In [ ]:
np.random.seed(5)
shuffle_index = np.random.permutation(df.shape[0])
training_ratio = int(len(shuffle_index) * 0.9)
train_index = shuffle_index[0:training_ratio]
valid_index = shuffle_index[training_ratio::]
training_data = df.iloc[train_index, :]
valid_data = df.iloc[valid_index, :]

In [ ]:
training_data.to_csv("data/train.txt", index=None, header=None)
valid_data.to_csv("data/valid.txt", index=None, header=None)

### test data

In [ ]:
import glob
import pandas as pd
import numpy as np
json_file_pathlist = glob.glob("data/test/*.png")
print(json_file_pathlist)

In [ ]:
len(json_file_pathlist)

In [ ]:
df = pd.DataFrame({"image_path": json_file_pathlist})
df.head()

In [ ]:
test_data = df.iloc[:]
test_data.to_csv("data/test.txt", index=None, header=None)

# YOLOv4

In [ ]:
%cd /content/gdrive/MyDrive/VRDL_HW2/darknet

/content/gdrive/MyDrive/VRDL_HW2/darknet


In [ ]:
"""
start training
"""
!chmod 777 darknet
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -map -dont_show 

In [ ]:
!chmod 777 darknet
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg backup/yolov4-obj_last.weights yolov4.conv.137 -dont_show -map

### validation data

In [ ]:
!chmod 777 darknet
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg backup/yolov4-obj_last.weights

### test data

In [ ]:
!chmod 777 ./darknet
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg backup/yolov4-obj_last.weights -dont_show -ext_output -out result.json < data/test.txt

# general result

In [ ]:
%cd /content/gdrive/My Drive/VRDL_HW2/darknet

In [ ]:
import os
import json
import cv2


def extract_img(json):
    try:
        temp = json['filename'].rsplit('/', 1)[-1]
        temp = int(temp.split('.', 1)[0])
        return temp
    except KeyError:
        return 0

with open('result.json', 'r') as reader:
    jf = json.loads(reader.read())

jf.sort(key=extract_img, reverse=False)

sub = []
cnt = 0
r_c = 'relative_coordinates'
for idx in jf:

    path = os.getcwd()
    img_path = os.path.join(path, idx['filename'])
    image = cv2.imread(img_path)
    h, w, _ = image.shape
    for o in idx['objects']:

        center_x = o[r_c]['center_x'] * w
        center_y = o[r_c]['center_y'] * h
        width = o[r_c]['width'] * w
        height = o[r_c]['height'] * h

        top = center_y - (height / 2)
        left = center_x - (width / 2)

        fname = idx['filename'].split('/')[2]
        fname = fname.split('.png')[0]
        dict_ = {}
        dict_['image_id'] = int(fname)
        dict_['score'] = o['confidence']
        dict_['category_id'] = int(o['name'])
        dict_['bbox'] = [left, top, width, height]

        sub.append(dict_)
    cnt += 1
    if cnt % 100 == 0:
        print(cnt)

print(len(sub))
ret = json.dumps(sub)
with open('answer.json', 'w') as fp:
    fp.write(ret)

# Test speed

In [2]:
%cd /content/gdrive/MyDrive/VRDL_HW2/darknet
!chmod 777 ./darknet
!chmod 777 ./test.sh

/content/gdrive/MyDrive/VRDL_HW2/darknet


In [5]:
import os
import subprocess
import time


def file_cmp(a):
    tmpa = a.lstrip('data/test/')
    tmpa = int(tmpa.rstrip('.png\n'))
    return tmpa

In [6]:
TEST_IMAGE_NUMBER = 100
all_test = open('data/test.txt', 'r')
save_test = open('data/test_speed.txt', 'w')
file_list = []
for item in all_test.readlines():
    file_list.append(item)
file_list.sort(key=lambda x: file_cmp(x))
all_test.close()
for i in range(TEST_IMAGE_NUMBER):
    save_test.write(file_list[i])
save_test.close()

In [15]:
cmd = ["./test.sh"]
start_time = time.time()
p = subprocess.Popen(cmd, stdout=subprocess.PIPE)
p.wait()
end_time = time.time()
print("\nInference time per image: ",
      (end_time - start_time) / TEST_IMAGE_NUMBER)


Inference time per image:  0.1481681799888611
